In [ ]:
#Import libraries
import pandas as pd
import numpy as np
import seaborn as sns
import datetime

In [ ]:
#Load data matrix
tickerdata = pd.read_excel(r'Data/Processed Data Matrix for Thesis.xlsx',engine='openpyxl',dtype=str)
tickerdata['CI'] = pd.to_numeric(tickerdata['CI'])
tickerdata['ISF'] = pd.to_numeric(tickerdata['ISF'])
tickerdata.head()

In [ ]:
#Reshape/melt data matrix to make time an attribute
transformedtickerdata = tickerdata.melt(id_vars=['Sharing Economy Firm', 'Incumbent Firms', 'Ticker',
       'Exchange', 'CI', 'Market Index', 'ISF', 'SP(Start(t=1))', 'SP(End(t=1))', 'SP(Start(t=3))',
       'SP(End(t=3))', 'SP(Start(t=6))', 'SP(End(t=6))', 'SP(Start(t=12))', 'SP(End(t=12))', 'SP(Start(t=24))', 'SP(End(t=24))', 'SP(Start(t=36))', 'SP(End(t=36))'],var_name='t',value_name='Date').drop(columns=['SP(Start(t=1))', 'SP(End(t=1))', 'SP(Start(t=3))',
       'SP(End(t=3))', 'SP(Start(t=6))', 'SP(End(t=6))', 'SP(Start(t=12))', 'SP(End(t=12))', 'SP(Start(t=24))', 'SP(End(t=24))', 'SP(Start(t=36))', 'SP(End(t=36))'])
transformedtickerdata['Ticker'] = transformedtickerdata['Ticker'].astype(str)
transformedtickerdata.tail()

In [ ]:
#Load equity data and transform
equitydata = pd.read_excel(r'Data/Bloomberg Equity Data.xlsx', engine='openpyxl',usecols=[0,1,3],sheet_name=None,dtype=str)
datalist = []
for ticker, data in equitydata.items():
    data['Ticker'] = str(ticker)
    datalist.append(data)
transformedequitydata = pd.concat(datalist,axis=0)
transformedequitydata.dropna(axis=1,how='all')
transformedequitydata['Last Price'] = pd.to_numeric(transformedequitydata['Last Price'])
transformedequitydata.head()

In [ ]:
#Load index data and transform
indexdata = pd.read_excel(r'Data/Bloomberg Index Data.xlsx', engine='openpyxl',usecols=[0,1],sheet_name='S&P500',dtype=str)
indexdata['Last Price'] = pd.to_numeric(indexdata['Last Price'])
indexdata['Date'] = pd.to_datetime(indexdata['Date'])
indexdata.head()

In [ ]:
#Merge ticker data with empty matrix
merged_data = transformedtickerdata.merge(transformedequitydata, how='left',left_on=['Date','Ticker'],right_on=['Date','Ticker']).dropna(axis=1,how='all').rename(columns={'Last Price':'SP(t)','Volume':'Volume(t)'})
merged_data['Volume(t)'] = pd.to_numeric(merged_data['Volume(t)'])
merged_data['Date'] = pd.to_datetime(merged_data['Date'])
merged_data.reset_index()
merged_data.head()

In [ ]:
#Merge index data with rest of data
merged_data=merged_data.merge(indexdata,how='left',left_on=['Date'],right_on=['Date']).rename(columns={'Last Price':'S&P500(t)'})
merged_data.head()

In [ ]:
#Re-Pivot Data
complete_data = merged_data.pivot(index=['Sharing Economy Firm','Incumbent Firms','Ticker','Exchange','CI','Market Index','ISF'],columns=['t'])
complete_data.head()

In [ ]:
#Drop Multindex
complete_data.columns = ['_'.join(col) for col in complete_data.columns]
complete_data.head()

In [ ]:
#Print column names to drop and rename
complete_data.columns

In [ ]:
#Drop Reset Index/Rename Columns/Drop Start Dates
complete_data = complete_data.reset_index().rename(columns={'SP(t)_End(t=1)':'SP_End(t=1)','SP(t)_End(t=3)':'SP_End(t=3)','SP(t)_End(t=6)':'SP_End(t=6)','SP(t)_End(t=12)':'SP_End(t=12)','SP(t)_End(t=24)':'SP_End(t=24)','SP(t)_End(t=36)':'SP_End(t=36)','SP(t)_Start(t=1)':'SP_Start(t=1)','SP(t)_Start(t=3)':'SP_Start(t=3)','SP(t)_Start(t=6)':'SP_Start(t=6)','SP(t)_Start(t=12)':'SP_Start(t=12)','SP(t)_Start(t=24)':'SP_Start(t=24)','SP(t)_Start(t=36)':'SP_Start(t=36)',
'Volume(t)_End(t=1)':'Volume_End(t=1)','Volume(t)_End(t=3)':'Volume_End(t=3)','Volume(t)_End(t=6)':'Volume_End(t=6)','Volume(t)_End(t=12)':'Volume_End(t=12)','Volume(t)_End(t=24)':'Volume_End(t=24)','Volume(t)_End(t=36)':'Volume_End(t=36)','Volume(t)_Start(t=1)':'Volume_Start(t=1)','Volume(t)_Start(t=3)':'Volume_Start(t=3)','Volume(t)_Start(t=6)':'Volume_Start(t=6)','Volume(t)_Start(t=12)':'Volume_Start(t=12)','Volume(t)_Start(t=24)':'Volume_Start(t=24)','Volume(t)_Start(t=36)':'Volume_Start(t=36)',
'S&P500(t)_End(t=1)':'S&P500_End(t=1)','S&P500(t)_End(t=3)':'S&P500_End(t=3)','S&P500(t)_End(t=6)':'S&P500_End(t=6)','S&P500(t)_End(t=12)':'S&P500_End(t=12)','S&P500(t)_End(t=24)':'S&P500_End(t=24)','S&P500(t)_End(t=36)':'S&P500_End(t=36)','S&P500(t)_Start(t=1)':'S&P500_Start(t=1)','S&P500(t)_Start(t=3)':'S&P500_Start(t=3)','S&P500(t)_Start(t=6)':'S&P500_Start(t=6)','S&P500(t)_Start(t=12)':'S&P500_Start(t=12)','S&P500(t)_Start(t=24)':'S&P500_Start(t=24)','S&P500(t)_Start(t=36)':'S&P500_Start(t=36)'}).drop(columns=['Date_Start(t=1)', 'Date_Start(t=12)', 'Date_Start(t=24)', 'Date_Start(t=3)', 'Date_Start(t=36)', 'Date_Start(t=6)'])
complete_data.head()

In [ ]:
#Add Columns for Share Price Growth Rate % for t=(1,3,6,12,24,36)
complete_data['% Change in Share Price (t=1)'] = ((complete_data['SP_End(t=1)']-complete_data['SP_Start(t=1)'])/complete_data['SP_Start(t=1)'])*100
complete_data['% Change in Share Price (t=3)'] = ((complete_data['SP_End(t=3)']-complete_data['SP_Start(t=3)'])/complete_data['SP_Start(t=3)'])*100
complete_data['% Change in Share Price (t=6)'] = ((complete_data['SP_End(t=6)']-complete_data['SP_Start(t=6)'])/complete_data['SP_Start(t=6)'])*100
complete_data['% Change in Share Price (t=12)'] = ((complete_data['SP_End(t=12)']-complete_data['SP_Start(t=12)'])/complete_data['SP_Start(t=12)'])*100
complete_data['% Change in Share Price (t=24)'] = ((complete_data['SP_End(t=24)']-complete_data['SP_Start(t=24)'])/complete_data['SP_Start(t=24)'])*100
complete_data['% Change in Share Price (t=36)'] = ((complete_data['SP_End(t=36)']-complete_data['SP_Start(t=36)'])/complete_data['SP_Start(t=36)'])*100
complete_data.head()

In [ ]:
#Add Columns for Volume Growth Rate % for t=(1,3,6,12,24,36)
complete_data['% Change in Volume (t=1)'] = ((complete_data['Volume_End(t=1)']-complete_data['Volume_Start(t=1)'])/complete_data['Volume_Start(t=1)'])*100
complete_data['% Change in Volume (t=3)'] = ((complete_data['Volume_End(t=3)']-complete_data['Volume_Start(t=3)'])/complete_data['Volume_Start(t=3)'])*100
complete_data['% Change in Volume (t=6)'] = ((complete_data['Volume_End(t=6)']-complete_data['Volume_Start(t=6)'])/complete_data['Volume_Start(t=6)'])*100
complete_data['% Change in Volume (t=12)'] = ((complete_data['Volume_End(t=12)']-complete_data['Volume_Start(t=12)'])/complete_data['Volume_Start(t=12)'])*100
complete_data['% Change in Volume (t=24)'] = ((complete_data['Volume_End(t=24)']-complete_data['Volume_Start(t=24)'])/complete_data['Volume_Start(t=24)'])*100
complete_data['% Change in Volume (t=36)'] = ((complete_data['Volume_End(t=36)']-complete_data['Volume_Start(t=36)'])/complete_data['Volume_Start(t=36)'])*100
complete_data.head()

In [ ]:
#Add Columns for S&P500 Growth Rate % for t=(1,3,6,12,24,36)
complete_data['% Change in S&P500 (t=1)'] = ((complete_data['S&P500_End(t=1)']-complete_data['S&P500_Start(t=1)'])/complete_data['S&P500_Start(t=1)'])*100
complete_data['% Change in S&P500 (t=3)'] = ((complete_data['S&P500_End(t=3)']-complete_data['S&P500_Start(t=3)'])/complete_data['S&P500_Start(t=3)'])*100
complete_data['% Change in S&P500 (t=6)'] = ((complete_data['S&P500_End(t=6)']-complete_data['S&P500_Start(t=6)'])/complete_data['S&P500_Start(t=6)'])*100
complete_data['% Change in S&P500 (t=12)'] = ((complete_data['S&P500_End(t=12)']-complete_data['S&P500_Start(t=12)'])/complete_data['S&P500_Start(t=12)'])*100
complete_data['% Change in S&P500 (t=24)'] = ((complete_data['S&P500_End(t=24)']-complete_data['S&P500_Start(t=24)'])/complete_data['S&P500_Start(t=24)'])*100
complete_data['% Change in S&P500 (t=36)'] = ((complete_data['S&P500_End(t=36)']-complete_data['S&P500_Start(t=36)'])/complete_data['S&P500_Start(t=36)'])*100
complete_data.head()

In [ ]:
#Convert Exchange to Dummy Variables for either NYSE, NASDAQ, or Other (OTCMKTS, TWSE, and NSE)
exchange_dummies = pd.get_dummies(complete_data['Exchange'])
complete_data = pd.concat([complete_data,exchange_dummies],axis=1)
complete_data['Other'] = complete_data['NYSEARCA']+complete_data['OTCMKTS']
complete_data['Sharing Firm Introduced'] = np.where(complete_data['ISF'] == 1, "Yes", "No")
complete_data['Relatedness with Sharing Firm'] = np.where(complete_data['CI'] == 1, "Closely Related", "Less Closely Related")
complete_data = complete_data.drop(columns=['NYSEARCA','OTCMKTS']).sort_values(by=['Sharing Economy Firm','CI'],ascending=[True, False])
complete_data.head()

In [ ]:
#Write complete data to Excel and CSV
with pd.ExcelWriter('Data/complete_data.xlsx',engine='openpyxl') as writer:
    complete_data.to_excel(writer,index=False)
complete_data.to_csv('Data/complete_data.csv',index=False)

In [ ]:
#Create STATA ready data
stata_data = complete_data[['CI', 'ISF', 'NASDAQ', 'NYSE', 'Other', '% Change in Volume (t=1)', '% Change in Volume (t=3)', '% Change in Volume (t=6)', '% Change in Volume (t=12)',  '% Change in Volume (t=24)',  '% Change in Volume (t=36)', '% Change in Share Price (t=1)', '% Change in Share Price (t=3)', '% Change in Share Price (t=6)', '% Change in Share Price (t=12)', '% Change in Share Price (t=24)', '% Change in Share Price (t=36)', '% Change in S&P500 (t=1)', '% Change in S&P500 (t=3)', '% Change in S&P500 (t=6)', '% Change in S&P500 (t=12)', '% Change in S&P500 (t=24)', '% Change in S&P500 (t=36)','Date_End(t=1)', 'Date_End(t=12)', 'Date_End(t=24)', 'Date_End(t=3)', 'Date_End(t=36)', 'Date_End(t=6)', 'Sharing Firm Introduced','Relatedness with Sharing Firm']].rename(columns={'% Change in Volume (t=1)':'one_m_diff_volume', '% Change in Volume (t=3)':'three_m_diff_volume', '% Change in Volume (t=6)':'six_m_diff_volume', '% Change in Volume (t=12)':'one_y_diff_volume', '% Change in Volume (t=24)':'two_y_diff_volume', '% Change in Volume (t=36)':'three_y_diff_volume', '% Change in Share Price (t=1)':'one_m_diff_shareprice', '% Change in Share Price (t=3)':'three_m_diff_shareprice', '% Change in Share Price (t=6)':'six_m_diff_shareprice', '% Change in Share Price (t=12)':'one_y_diff_shareprice', '% Change in Share Price (t=24)':'two_y_diff_shareprice', '% Change in Share Price (t=36)':'three_y_diff_shareprice', '% Change in S&P500 (t=1)':'one_m_diff_sp500', '% Change in S&P500 (t=3)':'three_m_diff_sp500', '% Change in S&P500 (t=6)':'six_m_diff_sp500', '% Change in S&P500 (t=12)':'one_y_diff_sp500', '% Change in S&P500 (t=24)':'two_y_diff_sp500', '% Change in S&P500 (t=36)':'three_y_diff_sp500'})
stata_data['Exchange'] = stata_data[['NASDAQ','NYSE','Other']].idxmax(axis=1)
stata_data.head()

In [ ]:
#Write STATA data to Excel and CSV
with pd.ExcelWriter('Data/stata_data.xlsx',engine='openpyxl') as writer:
    stata_data.to_excel(writer,index=False,)
stata_data.to_csv('Data/stata_data.csv',index=False)